# Learning Our Learners



## Libraries

In [1]:
import os
import tensorflow
import matplotlib.pyplot
from tensorflow.keras import datasets
import time
import numpy
import networkx
import itertools
import matplotlib.gridspec
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense, Flatten,  Dropout,  Activation, concatenate
from tensorflow.keras.models import Sequential

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# import helpers
from helpers import *

In this notebook, we will train several neural network models on the MNIST dataset. We will only use the digits 0 and 1 to simplify the problem. The data is normalized to be between 0 and 1 for better training performance. We will experiment with different model architectures, as defined by `layer_sizes_list`.

# Datasets

## Basic Dataset

In [4]:
# Load the necessary libraries
from tensorflow.keras.datasets import mnist

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data:
# We will only use two classes to simplify the problem
x_train = x_train[y_train < 2]
y_train = y_train[y_train < 2]
x_test = x_test[y_test < 2]
y_test = y_test[y_test < 2]

# Normalize the data to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0  


import numpy as np

# Set the seed for the random number generator
np.random.seed(0)

# Randomly select 100 indices from the range equal to the size of x_test
indices = np.random.choice(np.arange(x_test.shape[0]), size=100, replace=False)

# Use these indices to select samples from x_test and y_test
x_test_sample = x_test[indices]
y_test_sample = y_test[indices]

# select a random sample of 100 images from the x and y test sets with a sid of 42
x_test_sample = x_test[:100]
y_test_sample = y_test[:100]



## Data Sets With Noisy Pixel

In [5]:
modified_images_dict_sample = add_noisy_pixel_to_all_mnist(x_test_sample,noise_intensity=1)
# save_object(modified_images_dict, "one_pixel_modified_images_dict.pkl")
# modified_images_dict = load_object(filename="one_pixel_modified_images_dict.pkl")
# one_pixel_modified_images_dict = load_object(filename="one_pixel_modified_images_dict.pkl")

In [6]:
"""# tmp_str = ""
# for i in range(3):
#     for j in range(3):
#         tmp_str += f"x_test_px_{i}_{j} = modified_images_dict[({i},{j})]\n"
#         tmp_str += f"x_test_px_{28-i}_{j} = modified_images_dict[({i},{j})]\n"
#         tmp_str += f"x_test_px_{i}_{j} = modified_images_dict[({i},{j})]\n"
#         tmp_str += f"x_test_px_{i}_{j} = modified_images_dict[({i},{j})]\n"
# # print(tmp_str)
        


            
# x_test_px_0_0 = modified_images_dict[(0,0)]
# x_test_px_27_0 = modified_images_dict[(0,0)]
# x_test_px_27_27 = modified_images_dict[(0,0)]
# x_test_px_0_27 = modified_images_dict[(0,0)]
# x_test_px_0_1 = modified_images_dict[(0,1)]
# x_test_px_27_1 = modified_images_dict[(0,1)]
# x_test_px_27_27 = modified_images_dict[(0,1)]
# x_test_px_0_27 = modified_images_dict[(0,1)]
# x_test_px_0_2 = modified_images_dict[(0,2)]
# x_test_px_27_2 = modified_images_dict[(0,2)]
# x_test_px_27_26 = modified_images_dict[(0,2)]
# x_test_px_0_26 = modified_images_dict[(0,2)]
# x_test_px_1_0 = modified_images_dict[(1,0)]
# x_test_px_27_0 = modified_images_dict[(1,0)]
# x_test_px_27_27 = modified_images_dict[(1,0)]
# x_test_px_1_27 = modified_images_dict[(1,0)]
# x_test_px_1_1 = modified_images_dict[(1,1)]
# x_test_px_27_1 = modified_images_dict[(1,1)]
# x_test_px_27_27 = modified_images_dict[(1,1)]
# x_test_px_1_27 = modified_images_dict[(1,1)]
# x_test_px_1_2 = modified_images_dict[(1,2)]
# x_test_px_27_2 = modified_images_dict[(1,2)]
# x_test_px_27_26 = modified_images_dict[(1,2)]
# x_test_px_1_26 = modified_images_dict[(1,2)]
# x_test_px_2_0 = modified_images_dict[(2,0)]
# x_test_px_26_0 = modified_images_dict[(2,0)]
# x_test_px_26_27 = modified_images_dict[(2,0)]
# x_test_px_2_27 = modified_images_dict[(2,0)]
# x_test_px_2_1 = modified_images_dict[(2,1)]
# x_test_px_26_1 = modified_images_dict[(2,1)]
# x_test_px_26_27 = modified_images_dict[(2,1)]
# x_test_px_2_27 = modified_images_dict[(2,1)]
# x_test_px_2_2 = modified_images_dict[(2,2)]
# x_test_px_26_2 = modified_images_dict[(2,2)]
# x_test_px_26_26 = modified_images_dict[(2,2)]
# x_test_px_2_26 = modified_images_dict[(2,2)]

# add all of the above to a list
# x_test_px_list = [
#     x_test_px_0_0, x_test_px_27_0, x_test_px_27_27, x_test_px_0_27,
#     x_test_px_0_1, x_test_px_27_1, x_test_px_27_27, x_test_px_0_27,
#     x_test_px_0_2, x_test_px_27_2, x_test_px_27_26, x_test_px_0_26,
#     x_test_px_1_0, x_test_px_27_0, x_test_px_27_27, x_test_px_1_27,
#     x_test_px_1_1, x_test_px_27_1, x_test_px_27_27, x_test_px_1_27,
#     x_test_px_1_2, x_test_px_27_2, x_test_px_27_26, x_test_px_1_26,
#     x_test_px_2_0, x_test_px_26_0, x_test_px_26_27, x_test_px_2_27,
#     x_test_px_2_1, x_test_px_26_1, x_test_px_26_27, x_test_px_2_27,
#     x_test_px_2_2, x_test_px_26_2, x_test_px_26_26, x_test_px_2_26,
#     x_test_px_27_0, x_test_px_0_0, x_test_px_0_27, x_test_px_27_27,
#     x_test_px_27_1, x_test_px_0_1, x_test_px_0_27, x_test_px_27_27,
#     x_test_px_27_2, x_test_px_0_2, x_test_px_0_26, x_test_px_27_26,
#     x_test_px_27_0, x_test_px_1_0, x_test_px_1_27, x_test_px_27_27,
#     x_test_px_27_1, x_test_px_1_1, x_test_px_1_27, x_test_px_27_27,
#     x_test_px_27_2, x_test_px_1_2, x_test_px_1_26, x_test_px_27_26,
#     x_test_px_26_0, x_test_px_2_0, x_test_px_2_27, x_test_px_26_27,
#     x_test_px_26_1, x_test_px_2_1, x_test_px_2_27, x_test_px_26_27,
#     x_test_px_26_2, x_test_px_2_2, x_test_px_2_26, x_test_px_26_26
# ]

# clean_1_avgs_model1 = get_avg_outputs(model1,x_test[y_test==1])
# clean_0_avgs_model1 = get_avg_outputs(model1,x_test[y_test==0])
# clean_1_avgs_model2 = get_avg_outputs(model2,x_test[y_test==1])
# clean_0_avgs_model2 = get_avg_outputs(model2,x_test[y_test==0])
# save_object(clean_1_avgs_model1, "clean_1_avgs_model1.pkl")
# save_object(clean_0_avgs_model1, "clean_0_avgs_model1.pkl")
# save_object(clean_1_avgs_model2, "clean_1_avgs_model2.pkl")
# save_object(clean_0_avgs_model2, "clean_0_avgs_model2.pkl")
""";
# [model1, model2] = [tf.keras.models.load_model('models/model_n_[10, 6].keras'), tf.keras.models.load_model('models/model_n_[20, 20, 20, 10, 10, 10, 6].keras')]

# clean_1_avgs_model1_samp = get_avg_outputs(model1,x_test_sample[y_test_sample==1])
# clean_0_avgs_model1_samp = get_avg_outputs(model1,x_test_sample[y_test_sample==0])
# clean_1_avgs_model2_samp= get_avg_outputs(model2,x_test_sample[y_test_sample==1])
# clean_0_avgs_model2_samp= get_avg_outputs(model2,x_test_sample[y_test_sample==0])
# save_object(clean_1_avgs_model1_samp, "clean_1_avgs_model1_samp.pkl")
# save_object(clean_0_avgs_model1_samp, "clean_0_avgs_model1_samp.pkl")
# save_object(clean_1_avgs_model2_samp, "clean_1_avgs_model2_samp.pkl")
# save_object(clean_0_avgs_model2_samp, "clean_0_avgs_model2_samp.pkl")

In [6]:
models = {
    "model1": tf.keras.models.load_model('models/model_n_[10, 6].keras'),
    "model2": tf.keras.models.load_model('models/model_n_[20, 20, 20, 10, 10, 10, 6].keras')

}
[model1, model2] = [m1,m2] = [tf.keras.models.load_model('models/model_n_[10, 6].keras'), tf.keras.models.load_model('models/model_n_[20, 20, 20, 10, 10, 10, 6].keras')]

In [7]:
clean_1_avgs_model1_samp = load_object("clean_1_avgs_model1_samp.pkl")
clean_0_avgs_model1_samp = load_object("clean_0_avgs_model1_samp.pkl")
clean_1_avgs_model2_samp = load_object("clean_1_avgs_model2_samp.pkl")
clean_0_avgs_model2_samp = load_object("clean_0_avgs_model2_samp.pkl")

In [ ]:
# Directory where you want to save the images

numbers = [0,5, 10, 15, 20, 25]
models = ['m1', 'm2']
code_string = ""
# m = "m2"
for m in models:
  code_string+=f"# {m} model"
  n=m[1]
  for num1 in numbers:
      for num2 in numbers:
              block = f"""
# Combination: {num1} and {num2} for 
compare_model_avgs(model{n}, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[({num1},{num2})],outputs1=clean_0_avgs_model{n}_samp, outputs2=load_object("t_0s_avg_{m}_{num1}{num2}.pkl"),outputs3=load_object("t_0s_avg_{m}_{num1}{num2}_diff.pkl"), digit=0, indexes="{num1}{num2}")

compare_model_avgs(model{n}, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[({num1},{num2})],outputs1=clean_1_avgs_model{n}_samp, outputs2=load_object("t_1s_avg_{m}_{num1}{num2}.pkl"),outputs3=load_object("t_1s_avg_{m}_{num1}{num2}_diff.pkl"), digit=1,indexes="{num1}{num2}")

  """
              code_string += block

print(code_string)


In [47]:
# Loop through the x_test_px_list and models
for i in [5,10,15,20,25]:
    for j in [0]:
        
        print(f"({i},{j})")
        #time the function
        start = time.time()

        test1=modified_images_dict_sample[(i,j)][y_test_sample==1]
        test0=modified_images_dict_sample[(i,j)][y_test_sample==0]

        t_1s_avg_m1_xx = get_avg_outputs(model1,test1)
        t_1s_avg_m1_xx_diff = calculate_difference(clean_1_avgs_model1_samp,t_1s_avg_m1_xx)
        save_object(t_1s_avg_m1_xx, f"t_1s_avg_m1_{i}{j}.pkl")
        save_object(t_1s_avg_m1_xx_diff, f"t_1s_avg_m1_{i}{j}_diff.pkl")

        t_0s_avg_m1_xx = get_avg_outputs(model1,test0)
        t_0s_avg_m1_xx_diff = calculate_difference(clean_0_avgs_model1_samp,t_0s_avg_m1_xx)
        save_object(t_0s_avg_m1_xx, f"t_0s_avg_m1_{i}{j}.pkl")
        save_object(t_0s_avg_m1_xx_diff, f"t_0s_avg_m1_{i}{j}_diff.pkl")

        t_1s_avg_m2_xx = get_avg_outputs(model2,test1)
        t_1s_avg_m2_xx_diff = calculate_difference(clean_1_avgs_model2_samp,t_1s_avg_m2_xx)
        save_object(t_1s_avg_m2_xx, f"t_1s_avg_m2_{i}{j}.pkl")
        save_object(t_1s_avg_m2_xx_diff, f"t_1s_avg_m2_{i}{j}_diff.pkl")

        t_0s_avg_m2_xx = get_avg_outputs(model2,test0)
        t_0s_avg_m2_xx_diff = calculate_difference(clean_0_avgs_model2_samp,t_0s_avg_m2_xx)
        save_object(t_0s_avg_m2_xx, f"t_0s_avg_m2_{i}{j}.pkl")
        save_object(t_0s_avg_m2_xx_diff, f"t_0s_avg_m2_{i}{j}_diff.pkl")

        print( f"Time taken: {(time.time() - start) / 60} minutes")

(5,0)
Time taken: 0.3866238753000895 minutes
(10,0)
Time taken: 0.4849678874015808 minutes
(15,0)
Time taken: 0.33392792542775473 minutes
(20,0)
Time taken: 0.38032120068868003 minutes
(25,0)
Time taken: 0.3747716466585795 minutes


In [7]:
# clean_1_avgs_model1 = load_object("clean_1_avgs_model1.pkl")
# clean_0_avgs_model1 = load_object("clean_0_avgs_model1.pkl")
# clean_1_avgs_model2 = load_object("clean_1_avgs_model2.pkl")
# clean_0_avgs_model2 = load_object("clean_0_avgs_model2.pkl")
# # # Loop through the x_test_px_list and models
# for i in [14,15]:
#     for j in [6,8]:
#     # for j in [10]:
        
#         print(f"({i},{j})")
#         #time the function
#         start = time.time()

#         test1=modified_images_dict[(i,j)][y_test==1]
#         test0=modified_images_dict[(i,j)][y_test==0]

#         t_1s_avg_m1_xx = get_avg_outputs(model1,test1)
#         t_1s_avg_m1_xx_diff = calculate_difference(clean_1_avgs_model1,t_1s_avg_m1_xx)
#         save_object(t_1s_avg_m1_xx, f"t_1s_avg_m1_{i}{j}.pkl")
#         save_object(t_1s_avg_m1_xx_diff, f"t_1s_avg_m1_{i}{j}_diff.pkl")

#         t_0s_avg_m1_xx = get_avg_outputs(model1,test0)
#         t_0s_avg_m1_xx_diff = calculate_difference(clean_0_avgs_model1,t_0s_avg_m1_xx)
#         save_object(t_0s_avg_m1_xx, f"t_0s_avg_m1_{i}{j}.pkl")
#         save_object(t_0s_avg_m1_xx_diff, f"t_0s_avg_m1_{i}{j}_diff.pkl")

#         t_1s_avg_m2_xx = get_avg_outputs(model2,test1)
#         t_1s_avg_m2_xx_diff = calculate_difference(clean_1_avgs_model2,t_1s_avg_m2_xx)
#         save_object(t_1s_avg_m2_xx, f"t_1s_avg_m2_{i}{j}.pkl")
#         save_object(t_1s_avg_m2_xx_diff, f"t_1s_avg_m2_{i}{j}_diff.pkl")

#         t_0s_avg_m2_xx = get_avg_outputs(model2,test0)
#         t_0s_avg_m2_xx_diff = calculate_difference(clean_0_avgs_model2,t_0s_avg_m2_xx)
#         save_object(t_0s_avg_m2_xx, f"t_0s_avg_m2_{i}{j}.pkl")
#         save_object(t_0s_avg_m2_xx_diff, f"t_0s_avg_m2_{i}{j}_diff.pkl")

#         print( f"Time taken: {(time.time() - start) / 60} minutes")

(14,6)
Time taken: 4.62569884856542 minutes
(14,8)
Time taken: 6.1366503993670145 minutes
(15,6)
Time taken: 5.060393989086151 minutes
(15,8)
Time taken: 10.38217560450236 minutes


In [8]:
# m1 model
# Combination: 0 and 0 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,0)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_00.pkl"),outputs3=load_object("t_0s_avg_m1_00_diff.pkl"), digit=0, indexes="00")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,0)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_00.pkl"),outputs3=load_object("t_1s_avg_m1_00_diff.pkl"), digit=1,indexes="00")

  
# Combination: 0 and 5 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,5)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_05.pkl"),outputs3=load_object("t_0s_avg_m1_05_diff.pkl"), digit=0, indexes="05")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,5)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_05.pkl"),outputs3=load_object("t_1s_avg_m1_05_diff.pkl"), digit=1,indexes="05")

  
# Combination: 0 and 10 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,10)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_010.pkl"),outputs3=load_object("t_0s_avg_m1_010_diff.pkl"), digit=0, indexes="010")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,10)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_010.pkl"),outputs3=load_object("t_1s_avg_m1_010_diff.pkl"), digit=1,indexes="010")

  
# Combination: 0 and 15 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,15)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_015.pkl"),outputs3=load_object("t_0s_avg_m1_015_diff.pkl"), digit=0, indexes="015")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,15)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_015.pkl"),outputs3=load_object("t_1s_avg_m1_015_diff.pkl"), digit=1,indexes="015")

  
# Combination: 0 and 20 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,20)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_020.pkl"),outputs3=load_object("t_0s_avg_m1_020_diff.pkl"), digit=0, indexes="020")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,20)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_020.pkl"),outputs3=load_object("t_1s_avg_m1_020_diff.pkl"), digit=1,indexes="020")

  
# Combination: 0 and 25 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,25)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_025.pkl"),outputs3=load_object("t_0s_avg_m1_025_diff.pkl"), digit=0, indexes="025")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,25)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_025.pkl"),outputs3=load_object("t_1s_avg_m1_025_diff.pkl"), digit=1,indexes="025")

  
# Combination: 5 and 0 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,0)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_50.pkl"),outputs3=load_object("t_0s_avg_m1_50_diff.pkl"), digit=0, indexes="50")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,0)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_50.pkl"),outputs3=load_object("t_1s_avg_m1_50_diff.pkl"), digit=1,indexes="50")

  
# Combination: 5 and 5 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,5)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_55.pkl"),outputs3=load_object("t_0s_avg_m1_55_diff.pkl"), digit=0, indexes="55")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,5)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_55.pkl"),outputs3=load_object("t_1s_avg_m1_55_diff.pkl"), digit=1,indexes="55")

  
# Combination: 5 and 10 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,10)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_510.pkl"),outputs3=load_object("t_0s_avg_m1_510_diff.pkl"), digit=0, indexes="510")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,10)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_510.pkl"),outputs3=load_object("t_1s_avg_m1_510_diff.pkl"), digit=1,indexes="510")

  
# Combination: 5 and 15 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,15)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_515.pkl"),outputs3=load_object("t_0s_avg_m1_515_diff.pkl"), digit=0, indexes="515")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,15)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_515.pkl"),outputs3=load_object("t_1s_avg_m1_515_diff.pkl"), digit=1,indexes="515")

  
# Combination: 5 and 20 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,20)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_520.pkl"),outputs3=load_object("t_0s_avg_m1_520_diff.pkl"), digit=0, indexes="520")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,20)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_520.pkl"),outputs3=load_object("t_1s_avg_m1_520_diff.pkl"), digit=1,indexes="520")

  
# Combination: 5 and 25 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,25)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_525.pkl"),outputs3=load_object("t_0s_avg_m1_525_diff.pkl"), digit=0, indexes="525")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,25)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_525.pkl"),outputs3=load_object("t_1s_avg_m1_525_diff.pkl"), digit=1,indexes="525")

  
# Combination: 10 and 0 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,0)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_100.pkl"),outputs3=load_object("t_0s_avg_m1_100_diff.pkl"), digit=0, indexes="100")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,0)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_100.pkl"),outputs3=load_object("t_1s_avg_m1_100_diff.pkl"), digit=1,indexes="100")

  
# Combination: 10 and 5 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,5)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_105.pkl"),outputs3=load_object("t_0s_avg_m1_105_diff.pkl"), digit=0, indexes="105")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,5)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_105.pkl"),outputs3=load_object("t_1s_avg_m1_105_diff.pkl"), digit=1,indexes="105")

  
# Combination: 10 and 10 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,10)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_1010.pkl"),outputs3=load_object("t_0s_avg_m1_1010_diff.pkl"), digit=0, indexes="1010")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,10)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_1010.pkl"),outputs3=load_object("t_1s_avg_m1_1010_diff.pkl"), digit=1,indexes="1010")

  
# Combination: 10 and 15 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,15)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_1015.pkl"),outputs3=load_object("t_0s_avg_m1_1015_diff.pkl"), digit=0, indexes="1015")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,15)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_1015.pkl"),outputs3=load_object("t_1s_avg_m1_1015_diff.pkl"), digit=1,indexes="1015")

  
# Combination: 10 and 20 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,20)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_1020.pkl"),outputs3=load_object("t_0s_avg_m1_1020_diff.pkl"), digit=0, indexes="1020")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,20)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_1020.pkl"),outputs3=load_object("t_1s_avg_m1_1020_diff.pkl"), digit=1,indexes="1020")

  
# Combination: 10 and 25 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,25)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_1025.pkl"),outputs3=load_object("t_0s_avg_m1_1025_diff.pkl"), digit=0, indexes="1025")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,25)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_1025.pkl"),outputs3=load_object("t_1s_avg_m1_1025_diff.pkl"), digit=1,indexes="1025")

  
# Combination: 15 and 0 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,0)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_150.pkl"),outputs3=load_object("t_0s_avg_m1_150_diff.pkl"), digit=0, indexes="150")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,0)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_150.pkl"),outputs3=load_object("t_1s_avg_m1_150_diff.pkl"), digit=1,indexes="150")

  
# Combination: 15 and 5 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,5)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_155.pkl"),outputs3=load_object("t_0s_avg_m1_155_diff.pkl"), digit=0, indexes="155")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,5)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_155.pkl"),outputs3=load_object("t_1s_avg_m1_155_diff.pkl"), digit=1,indexes="155")

  
# Combination: 15 and 10 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,10)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_1510.pkl"),outputs3=load_object("t_0s_avg_m1_1510_diff.pkl"), digit=0, indexes="1510")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,10)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_1510.pkl"),outputs3=load_object("t_1s_avg_m1_1510_diff.pkl"), digit=1,indexes="1510")

  
# Combination: 15 and 15 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,15)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_1515.pkl"),outputs3=load_object("t_0s_avg_m1_1515_diff.pkl"), digit=0, indexes="1515")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,15)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_1515.pkl"),outputs3=load_object("t_1s_avg_m1_1515_diff.pkl"), digit=1,indexes="1515")

  
# Combination: 15 and 20 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,20)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_1520.pkl"),outputs3=load_object("t_0s_avg_m1_1520_diff.pkl"), digit=0, indexes="1520")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,20)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_1520.pkl"),outputs3=load_object("t_1s_avg_m1_1520_diff.pkl"), digit=1,indexes="1520")

  
# Combination: 15 and 25 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,25)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_1525.pkl"),outputs3=load_object("t_0s_avg_m1_1525_diff.pkl"), digit=0, indexes="1525")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,25)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_1525.pkl"),outputs3=load_object("t_1s_avg_m1_1525_diff.pkl"), digit=1,indexes="1525")

  
# Combination: 20 and 0 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,0)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_200.pkl"),outputs3=load_object("t_0s_avg_m1_200_diff.pkl"), digit=0, indexes="200")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,0)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_200.pkl"),outputs3=load_object("t_1s_avg_m1_200_diff.pkl"), digit=1,indexes="200")

  
# Combination: 20 and 5 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,5)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_205.pkl"),outputs3=load_object("t_0s_avg_m1_205_diff.pkl"), digit=0, indexes="205")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,5)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_205.pkl"),outputs3=load_object("t_1s_avg_m1_205_diff.pkl"), digit=1,indexes="205")

  
# Combination: 20 and 10 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,10)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_2010.pkl"),outputs3=load_object("t_0s_avg_m1_2010_diff.pkl"), digit=0, indexes="2010")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,10)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_2010.pkl"),outputs3=load_object("t_1s_avg_m1_2010_diff.pkl"), digit=1,indexes="2010")

  
# Combination: 20 and 15 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,15)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_2015.pkl"),outputs3=load_object("t_0s_avg_m1_2015_diff.pkl"), digit=0, indexes="2015")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,15)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_2015.pkl"),outputs3=load_object("t_1s_avg_m1_2015_diff.pkl"), digit=1,indexes="2015")

  
# Combination: 20 and 20 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,20)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_2020.pkl"),outputs3=load_object("t_0s_avg_m1_2020_diff.pkl"), digit=0, indexes="2020")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,20)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_2020.pkl"),outputs3=load_object("t_1s_avg_m1_2020_diff.pkl"), digit=1,indexes="2020")

  
# Combination: 20 and 25 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,25)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_2025.pkl"),outputs3=load_object("t_0s_avg_m1_2025_diff.pkl"), digit=0, indexes="2025")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,25)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_2025.pkl"),outputs3=load_object("t_1s_avg_m1_2025_diff.pkl"), digit=1,indexes="2025")

  
# Combination: 25 and 0 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,0)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_250.pkl"),outputs3=load_object("t_0s_avg_m1_250_diff.pkl"), digit=0, indexes="250")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,0)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_250.pkl"),outputs3=load_object("t_1s_avg_m1_250_diff.pkl"), digit=1,indexes="250")

  
# Combination: 25 and 5 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,5)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_255.pkl"),outputs3=load_object("t_0s_avg_m1_255_diff.pkl"), digit=0, indexes="255")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,5)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_255.pkl"),outputs3=load_object("t_1s_avg_m1_255_diff.pkl"), digit=1,indexes="255")

  
# Combination: 25 and 10 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,10)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_2510.pkl"),outputs3=load_object("t_0s_avg_m1_2510_diff.pkl"), digit=0, indexes="2510")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,10)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_2510.pkl"),outputs3=load_object("t_1s_avg_m1_2510_diff.pkl"), digit=1,indexes="2510")

  
# Combination: 25 and 15 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,15)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_2515.pkl"),outputs3=load_object("t_0s_avg_m1_2515_diff.pkl"), digit=0, indexes="2515")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,15)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_2515.pkl"),outputs3=load_object("t_1s_avg_m1_2515_diff.pkl"), digit=1,indexes="2515")

  
# Combination: 25 and 20 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,20)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_2520.pkl"),outputs3=load_object("t_0s_avg_m1_2520_diff.pkl"), digit=0, indexes="2520")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,20)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_2520.pkl"),outputs3=load_object("t_1s_avg_m1_2520_diff.pkl"), digit=1,indexes="2520")

  
# Combination: 25 and 25 for 
compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,25)],outputs1=clean_0_avgs_model1_samp, outputs2=load_object("t_0s_avg_m1_2525.pkl"),outputs3=load_object("t_0s_avg_m1_2525_diff.pkl"), digit=0, indexes="2525")

compare_model_avgs(model1, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,25)],outputs1=clean_1_avgs_model1_samp, outputs2=load_object("t_1s_avg_m1_2525.pkl"),outputs3=load_object("t_1s_avg_m1_2525_diff.pkl"), digit=1,indexes="2525")

  # m2 model
# Combination: 0 and 0 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,0)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_00.pkl"),outputs3=load_object("t_0s_avg_m2_00_diff.pkl"), digit=0, indexes="00")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,0)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_00.pkl"),outputs3=load_object("t_1s_avg_m2_00_diff.pkl"), digit=1,indexes="00")

  
# Combination: 0 and 5 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,5)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_05.pkl"),outputs3=load_object("t_0s_avg_m2_05_diff.pkl"), digit=0, indexes="05")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,5)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_05.pkl"),outputs3=load_object("t_1s_avg_m2_05_diff.pkl"), digit=1,indexes="05")

  
# Combination: 0 and 10 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,10)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_010.pkl"),outputs3=load_object("t_0s_avg_m2_010_diff.pkl"), digit=0, indexes="010")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,10)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_010.pkl"),outputs3=load_object("t_1s_avg_m2_010_diff.pkl"), digit=1,indexes="010")

  
# Combination: 0 and 15 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,15)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_015.pkl"),outputs3=load_object("t_0s_avg_m2_015_diff.pkl"), digit=0, indexes="015")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,15)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_015.pkl"),outputs3=load_object("t_1s_avg_m2_015_diff.pkl"), digit=1,indexes="015")

  
# Combination: 0 and 20 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,20)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_020.pkl"),outputs3=load_object("t_0s_avg_m2_020_diff.pkl"), digit=0, indexes="020")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,20)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_020.pkl"),outputs3=load_object("t_1s_avg_m2_020_diff.pkl"), digit=1,indexes="020")

  
# Combination: 0 and 25 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,25)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_025.pkl"),outputs3=load_object("t_0s_avg_m2_025_diff.pkl"), digit=0, indexes="025")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(0,25)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_025.pkl"),outputs3=load_object("t_1s_avg_m2_025_diff.pkl"), digit=1,indexes="025")

  
# Combination: 5 and 0 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,0)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_50.pkl"),outputs3=load_object("t_0s_avg_m2_50_diff.pkl"), digit=0, indexes="50")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,0)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_50.pkl"),outputs3=load_object("t_1s_avg_m2_50_diff.pkl"), digit=1,indexes="50")

  
# Combination: 5 and 5 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,5)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_55.pkl"),outputs3=load_object("t_0s_avg_m2_55_diff.pkl"), digit=0, indexes="55")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,5)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_55.pkl"),outputs3=load_object("t_1s_avg_m2_55_diff.pkl"), digit=1,indexes="55")

  
# Combination: 5 and 10 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,10)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_510.pkl"),outputs3=load_object("t_0s_avg_m2_510_diff.pkl"), digit=0, indexes="510")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,10)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_510.pkl"),outputs3=load_object("t_1s_avg_m2_510_diff.pkl"), digit=1,indexes="510")

  
# Combination: 5 and 15 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,15)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_515.pkl"),outputs3=load_object("t_0s_avg_m2_515_diff.pkl"), digit=0, indexes="515")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,15)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_515.pkl"),outputs3=load_object("t_1s_avg_m2_515_diff.pkl"), digit=1,indexes="515")

  
# Combination: 5 and 20 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,20)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_520.pkl"),outputs3=load_object("t_0s_avg_m2_520_diff.pkl"), digit=0, indexes="520")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,20)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_520.pkl"),outputs3=load_object("t_1s_avg_m2_520_diff.pkl"), digit=1,indexes="520")

  
# Combination: 5 and 25 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,25)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_525.pkl"),outputs3=load_object("t_0s_avg_m2_525_diff.pkl"), digit=0, indexes="525")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(5,25)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_525.pkl"),outputs3=load_object("t_1s_avg_m2_525_diff.pkl"), digit=1,indexes="525")

  
# Combination: 10 and 0 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,0)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_100.pkl"),outputs3=load_object("t_0s_avg_m2_100_diff.pkl"), digit=0, indexes="100")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,0)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_100.pkl"),outputs3=load_object("t_1s_avg_m2_100_diff.pkl"), digit=1,indexes="100")

  
# Combination: 10 and 5 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,5)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_105.pkl"),outputs3=load_object("t_0s_avg_m2_105_diff.pkl"), digit=0, indexes="105")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,5)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_105.pkl"),outputs3=load_object("t_1s_avg_m2_105_diff.pkl"), digit=1,indexes="105")

  
# Combination: 10 and 10 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,10)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_1010.pkl"),outputs3=load_object("t_0s_avg_m2_1010_diff.pkl"), digit=0, indexes="1010")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,10)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_1010.pkl"),outputs3=load_object("t_1s_avg_m2_1010_diff.pkl"), digit=1,indexes="1010")

  
# Combination: 10 and 15 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,15)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_1015.pkl"),outputs3=load_object("t_0s_avg_m2_1015_diff.pkl"), digit=0, indexes="1015")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,15)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_1015.pkl"),outputs3=load_object("t_1s_avg_m2_1015_diff.pkl"), digit=1,indexes="1015")

  
# Combination: 10 and 20 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,20)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_1020.pkl"),outputs3=load_object("t_0s_avg_m2_1020_diff.pkl"), digit=0, indexes="1020")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,20)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_1020.pkl"),outputs3=load_object("t_1s_avg_m2_1020_diff.pkl"), digit=1,indexes="1020")

  
# Combination: 10 and 25 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,25)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_1025.pkl"),outputs3=load_object("t_0s_avg_m2_1025_diff.pkl"), digit=0, indexes="1025")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(10,25)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_1025.pkl"),outputs3=load_object("t_1s_avg_m2_1025_diff.pkl"), digit=1,indexes="1025")

  
# Combination: 15 and 0 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,0)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_150.pkl"),outputs3=load_object("t_0s_avg_m2_150_diff.pkl"), digit=0, indexes="150")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,0)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_150.pkl"),outputs3=load_object("t_1s_avg_m2_150_diff.pkl"), digit=1,indexes="150")

  
# Combination: 15 and 5 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,5)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_155.pkl"),outputs3=load_object("t_0s_avg_m2_155_diff.pkl"), digit=0, indexes="155")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,5)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_155.pkl"),outputs3=load_object("t_1s_avg_m2_155_diff.pkl"), digit=1,indexes="155")

  
# Combination: 15 and 10 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,10)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_1510.pkl"),outputs3=load_object("t_0s_avg_m2_1510_diff.pkl"), digit=0, indexes="1510")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,10)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_1510.pkl"),outputs3=load_object("t_1s_avg_m2_1510_diff.pkl"), digit=1,indexes="1510")

  
# Combination: 15 and 15 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,15)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_1515.pkl"),outputs3=load_object("t_0s_avg_m2_1515_diff.pkl"), digit=0, indexes="1515")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,15)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_1515.pkl"),outputs3=load_object("t_1s_avg_m2_1515_diff.pkl"), digit=1,indexes="1515")

  
# Combination: 15 and 20 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,20)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_1520.pkl"),outputs3=load_object("t_0s_avg_m2_1520_diff.pkl"), digit=0, indexes="1520")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,20)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_1520.pkl"),outputs3=load_object("t_1s_avg_m2_1520_diff.pkl"), digit=1,indexes="1520")

  
# Combination: 15 and 25 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,25)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_1525.pkl"),outputs3=load_object("t_0s_avg_m2_1525_diff.pkl"), digit=0, indexes="1525")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(15,25)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_1525.pkl"),outputs3=load_object("t_1s_avg_m2_1525_diff.pkl"), digit=1,indexes="1525")

  
# Combination: 20 and 0 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,0)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_200.pkl"),outputs3=load_object("t_0s_avg_m2_200_diff.pkl"), digit=0, indexes="200")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,0)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_200.pkl"),outputs3=load_object("t_1s_avg_m2_200_diff.pkl"), digit=1,indexes="200")

  
# Combination: 20 and 5 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,5)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_205.pkl"),outputs3=load_object("t_0s_avg_m2_205_diff.pkl"), digit=0, indexes="205")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,5)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_205.pkl"),outputs3=load_object("t_1s_avg_m2_205_diff.pkl"), digit=1,indexes="205")

  
# Combination: 20 and 10 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,10)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_2010.pkl"),outputs3=load_object("t_0s_avg_m2_2010_diff.pkl"), digit=0, indexes="2010")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,10)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_2010.pkl"),outputs3=load_object("t_1s_avg_m2_2010_diff.pkl"), digit=1,indexes="2010")

  
# Combination: 20 and 15 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,15)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_2015.pkl"),outputs3=load_object("t_0s_avg_m2_2015_diff.pkl"), digit=0, indexes="2015")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,15)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_2015.pkl"),outputs3=load_object("t_1s_avg_m2_2015_diff.pkl"), digit=1,indexes="2015")

  
# Combination: 20 and 20 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,20)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_2020.pkl"),outputs3=load_object("t_0s_avg_m2_2020_diff.pkl"), digit=0, indexes="2020")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,20)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_2020.pkl"),outputs3=load_object("t_1s_avg_m2_2020_diff.pkl"), digit=1,indexes="2020")

  
# Combination: 20 and 25 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,25)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_2025.pkl"),outputs3=load_object("t_0s_avg_m2_2025_diff.pkl"), digit=0, indexes="2025")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(20,25)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_2025.pkl"),outputs3=load_object("t_1s_avg_m2_2025_diff.pkl"), digit=1,indexes="2025")

  
# Combination: 25 and 0 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,0)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_250.pkl"),outputs3=load_object("t_0s_avg_m2_250_diff.pkl"), digit=0, indexes="250")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,0)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_250.pkl"),outputs3=load_object("t_1s_avg_m2_250_diff.pkl"), digit=1,indexes="250")

  
# Combination: 25 and 5 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,5)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_255.pkl"),outputs3=load_object("t_0s_avg_m2_255_diff.pkl"), digit=0, indexes="255")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,5)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_255.pkl"),outputs3=load_object("t_1s_avg_m2_255_diff.pkl"), digit=1,indexes="255")

  
# Combination: 25 and 10 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,10)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_2510.pkl"),outputs3=load_object("t_0s_avg_m2_2510_diff.pkl"), digit=0, indexes="2510")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,10)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_2510.pkl"),outputs3=load_object("t_1s_avg_m2_2510_diff.pkl"), digit=1,indexes="2510")

  
# Combination: 25 and 15 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,15)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_2515.pkl"),outputs3=load_object("t_0s_avg_m2_2515_diff.pkl"), digit=0, indexes="2515")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,15)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_2515.pkl"),outputs3=load_object("t_1s_avg_m2_2515_diff.pkl"), digit=1,indexes="2515")

  
# Combination: 25 and 20 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,20)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_2520.pkl"),outputs3=load_object("t_0s_avg_m2_2520_diff.pkl"), digit=0, indexes="2520")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,20)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_2520.pkl"),outputs3=load_object("t_1s_avg_m2_2520_diff.pkl"), digit=1,indexes="2520")

  
# Combination: 25 and 25 for 
compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,25)],outputs1=clean_0_avgs_model2_samp, outputs2=load_object("t_0s_avg_m2_2525.pkl"),outputs3=load_object("t_0s_avg_m2_2525_diff.pkl"), digit=0, indexes="2525")

compare_model_avgs(model2, X=x_test_sample, y=y_test_sample, X_noisy=modified_images_dict_sample[(25,25)],outputs1=clean_1_avgs_model2_samp, outputs2=load_object("t_1s_avg_m2_2525.pkl"),outputs3=load_object("t_1s_avg_m2_2525_diff.pkl"), digit=1,indexes="2525")

  


In [13]:
"""MOVE PIKLES TO DIR"""
import os
import shutil

# Directory where you want to search for .pkl files
source_dir = 'C:/Users/avivg/My Code Scripts/ArielJaffe/1-poc-watch-nn-wheights'

# Directory where you want to move the .pkl files
target_dir = 'C:/Users/avivg/My Code Scripts/ArielJaffe/1-poc-watch-nn-wheights/Pikles'
os.makedirs(target_dir, exist_ok=True)

# Find and move all .pkl files
for filename in os.listdir(source_dir):
    if filename.endswith('.pkl'):
        source_path = os.path.join(source_dir, filename)
        target_path = os.path.join(target_dir, filename)
        shutil.move(source_path, target_path)
        print(f"Moved {filename} to {target_dir}")


Moved clean_0_avgs_model1.pkl to C:/Users/avivg/My Code Scripts/ArielJaffe/1-poc-watch-nn-wheights/Pikles
Moved clean_0_avgs_model1_samp.pkl to C:/Users/avivg/My Code Scripts/ArielJaffe/1-poc-watch-nn-wheights/Pikles
Moved clean_0_avgs_model2.pkl to C:/Users/avivg/My Code Scripts/ArielJaffe/1-poc-watch-nn-wheights/Pikles
Moved clean_0_avgs_model2_samp.pkl to C:/Users/avivg/My Code Scripts/ArielJaffe/1-poc-watch-nn-wheights/Pikles
Moved clean_1_avgs_model1.pkl to C:/Users/avivg/My Code Scripts/ArielJaffe/1-poc-watch-nn-wheights/Pikles
Moved clean_1_avgs_model1_samp.pkl to C:/Users/avivg/My Code Scripts/ArielJaffe/1-poc-watch-nn-wheights/Pikles
Moved clean_1_avgs_model2.pkl to C:/Users/avivg/My Code Scripts/ArielJaffe/1-poc-watch-nn-wheights/Pikles
Moved clean_1_avgs_model2_samp.pkl to C:/Users/avivg/My Code Scripts/ArielJaffe/1-poc-watch-nn-wheights/Pikles
Moved test0avg1.pkl to C:/Users/avivg/My Code Scripts/ArielJaffe/1-poc-watch-nn-wheights/Pikles
Moved test0avg1pix00.pkl to C:/Users

In [ ]:
# 